In [5]:
#pip install opencv-python mediapipe numpy
#!pip install pygame

In [1]:
import cv2
import mediapipe as mp
import random
import atexit
import csv
import os
import pygame
import numpy as np
import time
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

csv_file_path = "hand_cricket_data.csv"

def detectHandsLandmarks(image, hands, draw=True, display=True):
    output_image = image.copy()
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks and draw:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image=output_image, landmark_list=hand_landmarks, connections=mp_hands.HAND_CONNECTIONS, landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2), connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    if display:
        plt.figure(figsize=[15, 15])
        plt.subplot(121);
        plt.imshow(image[:, :, ::-1]);
        plt.title("Original Image");
        plt.axis('off');
        plt.subplot(122);
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output");
        plt.axis('off');
    else:
        return output_image, results

def countFingers(image, results, draw=True, display=True):
    height, width, _ = image.shape
    output_image = image.copy()
    count = {'RIGHT': 0, 'LEFT': 0}
    fingers_tips_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    fingers_statuses = {'RIGHT_THUMB': False, 'RIGHT_INDEX': False, 'RIGHT_MIDDLE': False, 'RIGHT_RING': False, 'RIGHT_PINKY': False, 'LEFT_THUMB': False, 'LEFT_INDEX': False, 'LEFT_MIDDLE': False, 'LEFT_RING': False, 'LEFT_PINKY': False}
    for hand_index, hand_info in enumerate(results.multi_handedness):
        hand_label = hand_info.classification[0].label
        hand_landmarks = results.multi_hand_landmarks[hand_index]
        for tip_index in fingers_tips_ids:
            finger_name = tip_index.name.split("_")[0]
            if hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index - 2].y:
                fingers_statuses[hand_label.upper() + "_" + finger_name] = True
                count[hand_label.upper()] += 1
        thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
        thumb_mcp_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP - 2].x
        if (hand_label == 'Right' and (thumb_tip_x < thumb_mcp_x)) or (
                hand_label == 'Left' and (thumb_tip_x > thumb_mcp_x)):
            fingers_statuses[hand_label.upper() + "_THUMB"] = True
            count[hand_label.upper()] += 1
    if draw:
        cv2.putText(output_image, " Current Score: ", (10, 25), cv2.FONT_HERSHEY_COMPLEX, 1, (20, 255, 155), 2)
        cv2.putText(output_image, str(sum(count.values())), (width // 2 - 150, 240), cv2.FONT_HERSHEY_SIMPLEX, 8.9, (20, 255, 155), 10, 10)
    if display:
        plt.figure(figsize=[10, 10])
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output Image");
        plt.axis('off');
    else:
        return output_image, fingers_statuses, count

def computer_move():
    return random.randint(1, 6)

def save_to_csv(player_move, computer_score):
    mode = 'a' if os.path.exists(csv_file_path) else 'w'

    with open(csv_file_path, mode, newline='') as csvfile:
        fieldnames = ['Player_Move', 'Computer_Score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if mode == 'w':
            writer.writeheader()

        writer.writerow({'Player_Move': player_move, 'Computer_Score': computer_score})

def delete_csv_file():
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"Deleted CSV file: {csv_file_path}")

def play_hand_cricket():
    choice = input("Enter 'o' for odd or 'e' for even: ")
    if choice == 'e':
        choice = 'Even'
    elif choice == 'o':
        choice = 'Odd'
    else:
        print('Invalid Input Given \nShutting Down')
        return
    
    camera_video = cv2.VideoCapture(0)
    camera_video.set(3, 1280)
    camera_video.set(4, 960)

    paused = False
    start_time = time.time()
    pause_interval = 5
    countdown_timer = pause_interval
    
    player_move = 0
    computer_score = 0
    player_move_updated = False
    
    cv2.namedWindow('Hand Cricket', cv2.WINDOW_NORMAL)

    while camera_video.isOpened():
        elapsed_time = time.time() - start_time
        if elapsed_time >= pause_interval:
            paused = not paused
            start_time = time.time()
            if paused:
                countdown_timer = pause_interval
            print("Paused" if paused else "Resumed")

        if not paused:
            ret, frame = camera_video.read()

            if not ret:
                print("Error reading frame")
                continue

            frame = cv2.flip(frame, 1)
            frame, results = detectHandsLandmarks(frame, hands_videos, display=False)

            if results.multi_hand_landmarks:
                frame, fingers_statuses, count = countFingers(frame, results, display=False)

                player_move = sum(count.values())
                if player_move > 0:
                    player_move_updated = True

        k = cv2.waitKey(1) & 0xFF

        if k == ord('p') or paused:
            if player_move_updated:
                computer_score = computer_move()
                print("Player's move:", player_move)
                print("Computer's move:", computer_score)
                player_move_updated = False
                save_to_csv(player_move, computer_score)

        if k == 27: # Press ESC to Exit
            delete_csv_file()  
            break

        if paused:
            cv2.putText(frame, f"Player's Move: {player_move}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Computer's Move: {computer_score}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if 'choice' in locals():
                cv2.putText(frame, f"Your Choice: {choice}", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if countdown_timer > 0:
                countdown_timer -= 1

        elif not paused:
            if elapsed_time < pause_interval:
                cv2.putText(frame, f"Countdown: {int(pause_interval - elapsed_time)}", (10, 1250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            else:
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Hand Cricket', frame)

    camera_video.release()
    print('Video Released')

    cv2.destroyAllWindows()
    for i in range(1, 5):
        cv2.waitKey(1)
    print('Window Destroyed')

if __name__ == "__main__":
    play_hand_cricket()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


pygame 2.5.2 (SDL 2.28.3, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


Enter 'o' for odd or 'e' for even:  e


Paused
Player's move: 3
Computer's move: 3
Resumed
Paused
Player's move: 6
Computer's move: 6
Resumed
Paused
Player's move: 8
Computer's move: 2
Resumed
Paused
Player's move: 4
Computer's move: 6
Resumed
Paused
Player's move: 3
Computer's move: 4
Resumed
Paused
Player's move: 3
Computer's move: 4
Resumed
Paused
Player's move: 1
Computer's move: 2
Deleted CSV file: hand_cricket_data.csv
Video Released
Window Destroyed


In [7]:
import cv2
import mediapipe as mp
import random
import csv
import os
import time
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

csv_file_path = "hand_cricket_data.csv"

def detectHandsLandmarks(image, hands, draw=True, display=True):
    output_image = image.copy()
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks and draw:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image=output_image, landmark_list=hand_landmarks, connections=mp_hands.HAND_CONNECTIONS, landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2), connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    if display:
        plt.figure(figsize=[15, 15])
        plt.subplot(121);
        plt.imshow(image[:, :, ::-1]);
        plt.title("Original Image");
        plt.axis('off');
        plt.subplot(122);
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output");
        plt.axis('off');
    else:
        return output_image, results

def countFingers(image, results, draw=True, display=True):
    height, width, _ = image.shape
    output_image = image.copy()
    count = {'RIGHT': 0, 'LEFT': 0}
    fingers_tips_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    fingers_statuses = {'RIGHT_THUMB': False, 'RIGHT_INDEX': False, 'RIGHT_MIDDLE': False, 'RIGHT_RING': False, 'RIGHT_PINKY': False, 'LEFT_THUMB': False, 'LEFT_INDEX': False, 'LEFT_MIDDLE': False, 'LEFT_RING': False, 'LEFT_PINKY': False}
    for hand_index, hand_info in enumerate(results.multi_handedness):
        hand_label = hand_info.classification[0].label
        hand_landmarks = results.multi_hand_landmarks[hand_index]
        for tip_index in fingers_tips_ids:
            finger_name = tip_index.name.split("_")[0]
            if hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index - 2].y:
                fingers_statuses[hand_label.upper() + "_" + finger_name] = True
                count[hand_label.upper()] += 1
        thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
        thumb_mcp_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP - 2].x
        if (hand_label == 'Right' and (thumb_tip_x < thumb_mcp_x)) or (
                hand_label == 'Left' and (thumb_tip_x > thumb_mcp_x)):
            fingers_statuses[hand_label.upper() + "_THUMB"] = True
            count[hand_label.upper()] += 1
    if draw:
        cv2.putText(output_image, " Current Score: ", (10, 25), cv2.FONT_HERSHEY_COMPLEX, 1, (20, 255, 155), 2)
        cv2.putText(output_image, str(sum(count.values())), (width // 2 - 150, 240), cv2.FONT_HERSHEY_SIMPLEX, 8.9, (20, 255, 155), 10, 10)
    if display:
        plt.figure(figsize=[10, 10])
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output Image");
        plt.axis('off');
    else:
        return output_image, fingers_statuses, count

def computer_move():
    return random.randint(1, 6)

def computer_choice():
    return random.choice(['bat', 'ball'])

def save_to_csv(player_move, computer_score):
    mode = 'a' if os.path.exists(csv_file_path) else 'w'

    with open(csv_file_path, mode, newline='') as csvfile:
        fieldnames = ['Player_Move', 'Computer_Score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if mode == 'w':
            writer.writeheader()

        writer.writerow({'Player_Move': player_move, 'Computer_Score': computer_score})

def delete_csv_file():
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"Deleted CSV file: {csv_file_path}")

def get_row_from_csv(row_number):
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            if i == row_number:
                return int(row['Player_Move']), int(row['Computer_Score'])
    return 0, 0  # Default values if the CSV file is empty

def play_hand_cricket():
    choice = input("Enter 'o' for odd or 'e' for even: ")
    if choice == 'e':
        choice = 'Even'
    elif choice == 'o':
        choice = 'Odd'
    else:
        print('Invalid Input Given \nShutting Down')
        return
    
    camera_video = cv2.VideoCapture(0)
    camera_video.set(3, 1280)
    camera_video.set(4, 960)

    paused = False
    choice_winner = False
    
    start_time = time.time()
    pause_interval = 5
    countdown_timer = pause_interval
    
    player_move_updated = False
    
    cv2.namedWindow('Hand Cricket', cv2.WINDOW_NORMAL)

    while camera_video.isOpened():
        elapsed_time = time.time() - start_time
        if elapsed_time >= pause_interval:
            paused = not paused
            start_time = time.time()
            if paused:
                countdown_timer = pause_interval
            print("Paused" if paused else "Resumed")

        if not paused:
            ret, frame = camera_video.read()

            if not ret:
                print("Error reading frame")
                continue

            frame = cv2.flip(frame, 1)
            frame, results = detectHandsLandmarks(frame, hands_videos, display=False)

            if results.multi_hand_landmarks:
                frame, fingers_statuses, count = countFingers(frame, results, display=False)

                player_move = sum(count.values())
                if player_move > 0:
                    player_move_updated = True

        k = cv2.waitKey(1) & 0xFF

        if k == ord('p') or paused:
            if player_move_updated:
                computer_score = computer_move()

                print("Player's move:", player_move)
                print("Computer's move:", computer_score)

                player_move_updated = False
                save_to_csv(player_move, computer_score)
                if not choice_winner:

                    # Get the initial values from the first row of the CSV file
                    initial_pmove, initial_cscore = get_row_from_csv(0)

                    total_score = initial_pmove + initial_cscore

                    print("Total Score:", total_score)

                    # Check if the total score is even or odd
                    is_total_score_even = total_score % 2 == 0

                    # Compare with the user's choice
                    if (choice == 'Even' and is_total_score_even) or (choice == 'Odd' and not is_total_score_even):
                        print("Player Wins!")
                        p = input('Enter "bat" for Batting and "ball" for Balling')
                        if p == 'bat':
                            player_choice = p
                            print('Player has chosen Batting')
                        elif p == 'ball':
                            player_choice = p
                            print('Player has chosen Balling')
                        else:
                            print('Invalid Input Given\nShutting Down')
                            return
                            
                    else:
                        print("Computer Wins!")
                        c = computer_choice()
                        if c == 'bat':
                            print('Computer has chosen Batting')

                        else:
                            print('Computer has chosen Balling')
                    
                    choice_winner = True
                        
        if k == 27:  # Press ESC to Exit
            delete_csv_file()  
            break

        if paused:
            cv2.putText(frame, f"Player's Move: {player_move}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Computer's Move: {computer_score}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            if 'choice' in locals():
                cv2.putText(frame, f"Your Choice: {choice}", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if countdown_timer > 0:
                countdown_timer -= 1

        elif not paused:
            if elapsed_time < pause_interval:
                cv2.putText(frame, f"Countdown: {int(pause_interval - elapsed_time)}", (10, 1250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            else:
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Hand Cricket', frame)

    camera_video.release()
    print('Video Released')

    cv2.destroyAllWindows()
    for i in range(1, 5):
        cv2.waitKey(1)
    print('Window Destroyed')

if __name__ == "__main__":
    play_hand_cricket()


Enter 'o' for odd or 'e' for even:  o


Paused
Player's move: 2
Computer's move: 5
Total Score: 7
Player Wins!


Enter "bat" for Batting and "ball" for Balling ball


Player has chosen Balling
Resumed
Paused
Deleted CSV file: hand_cricket_data.csv
Video Released
Window Destroyed


In [ ]:
import cv2
import mediapipe as mp
import random
import csv
import os
import time
import matplotlib.pyplot as plt
import pygame

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

csv_file_path = "hand_cricket_data.csv"

def detectHandsLandmarks(image, hands, draw=True, display=True):
    output_image = image.copy()
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks and draw:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image=output_image, landmark_list=hand_landmarks, connections=mp_hands.HAND_CONNECTIONS, landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2), connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    if display:
        plt.figure(figsize=[15, 15])
        plt.subplot(121);
        plt.imshow(image[:, :, ::-1]);
        plt.title("Original Image");
        plt.axis('off');
        plt.subplot(122);
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output");
        plt.axis('off');
    else:
        return output_image, results

def countFingers(image, results, draw=True, display=True):
    height, width, _ = image.shape
    output_image = image.copy()
    count = {'RIGHT': 0, 'LEFT': 0}
    fingers_tips_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    fingers_statuses = {'RIGHT_THUMB': False, 'RIGHT_INDEX': False, 'RIGHT_MIDDLE': False, 'RIGHT_RING': False, 'RIGHT_PINKY': False, 'LEFT_THUMB': False, 'LEFT_INDEX': False, 'LEFT_MIDDLE': False, 'LEFT_RING': False, 'LEFT_PINKY': False}
    for hand_index, hand_info in enumerate(results.multi_handedness):
        hand_label = hand_info.classification[0].label
        hand_landmarks = results.multi_hand_landmarks[hand_index]
        for tip_index in fingers_tips_ids:
            finger_name = tip_index.name.split("_")[0]
            if hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index - 2].y:
                fingers_statuses[hand_label.upper() + "_" + finger_name] = True
                count[hand_label.upper()] += 1
        thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
        thumb_mcp_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP - 2].x
        if (hand_label == 'Right' and (thumb_tip_x < thumb_mcp_x)) or (
                hand_label == 'Left' and (thumb_tip_x > thumb_mcp_x)):
            fingers_statuses[hand_label.upper() + "_THUMB"] = True
            count[hand_label.upper()] += 1
    if draw:
        cv2.putText(output_image, " Current Score: ", (10, 25), cv2.FONT_HERSHEY_COMPLEX, 1, (20, 255, 155), 2)
        cv2.putText(output_image, str(sum(count.values())), (width // 2 - 150, 240), cv2.FONT_HERSHEY_SIMPLEX, 8.9, (20, 255, 155), 10, 10)
    if display:
        plt.figure(figsize=[10, 10])
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output Image");
        plt.axis('off');
    else:
        return output_image, fingers_statuses, count

def computer_move():
    return random.randint(1, 6)

def computer_choice():
    return random.choice(['bat', 'ball'])

def save_to_csv(player_move, computer_score):
    mode = 'a' if os.path.exists(csv_file_path) else 'w'

    with open(csv_file_path, mode, newline='') as csvfile:
        fieldnames = ['Player_Move', 'Computer_Score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if mode == 'w':
            writer.writeheader()

        writer.writerow({'Player_Move': player_move, 'Computer_Score': computer_score})

def delete_csv_file():
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"Deleted CSV file: {csv_file_path}")

def get_row_from_csv(row_number):
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            if i == row_number:
                return int(row['Player_Move']), int(row['Computer_Score'])
    return 0, 0  # Default values if the CSV file is empty

def play_hand_cricket():
    choice = input("Enter 'o' for odd or 'e' for even: ")
    if choice == 'e':
        choice = 'Even'
        print('Player has chosen', choice)
    elif choice == 'o':
        choice = 'Odd'
        print('Player has chosen', choice)
    else:
        print('Invalid Input Given \nShutting Down')
        return
    
    camera_video = cv2.VideoCapture(0)
    camera_video.set(3, 1280)
    camera_video.set(4, 960)

    paused = False
    choice_winner = False
    target = 0
    target_set = False
    target_reached = False
    target_iteration = True
    
    player_choice = ''
    compChoice = ''
    
    start_time = time.time()
    pause_interval = 5
    countdown_timer = pause_interval
    
    player_move_updated = False
    
    cv2.namedWindow('Hand Cricket', cv2.WINDOW_NORMAL)

    while camera_video.isOpened():
            
        elapsed_time = time.time() - start_time
        if elapsed_time >= pause_interval:
            paused = not paused
            start_time = time.time()
            if paused:
                countdown_timer = pause_interval
            print("Paused" if paused else "Resumed")

        if not paused:
            ret, frame = camera_video.read()

            if not ret:
                print("Error reading frame")
                continue

            frame = cv2.flip(frame, 1)
            frame, results = detectHandsLandmarks(frame, hands_videos, display=False)

            if results.multi_hand_landmarks:
                frame, fingers_statuses, count = countFingers(frame, results, display=False)

                player_move = sum(count.values())
                if player_move > 0:
                    player_move_updated = True

        k = cv2.waitKey(1) & 0xFF

        if k == ord('p') or paused:
            if player_move_updated:
                computer_score = computer_move()

                print("Player's move:", player_move)
                print("Computer's move:", computer_score)

                player_move_updated = False
                save_to_csv(player_move, computer_score)
                if not choice_winner:

                    # Get the initial values from the first row of the CSV file
                    initial_pmove, initial_cscore = get_row_from_csv(0)

                    total_score = initial_pmove + initial_cscore

                    print("Total Score:", total_score)

                    # Check if the total score is even or odd
                    is_total_score_even = total_score % 2 == 0

                    # Compare with the user's choice
                    if (choice == 'Even' and is_total_score_even) or (choice == 'Odd' and not is_total_score_even):
                        print("Player Wins!")
                        p = input('Enter "bat" for Batting and "ball" for Balling: ')
                        if p == 'bat':
                            player_choice = p
                            print('Player has chosen Batting')
                        elif p == 'ball':
                            player_choice = p
                            print('Player has chosen Balling')
                        else:
                            print('Invalid Input Given\nShutting Down')
                            cv2.destroyAllWindows()
                            for i in range(1, 5):
                                cv2.waitKey(1)
                            print('Window Destroyed')
                            return     
                    else:
                        print("Computer Wins!")
                        compChoice = computer_choice()
                        if compChoice == 'bat':
                            print('Computer has chosen Batting')
                        else:
                            print('Computer has chosen Balling')
                    
                    choice_winner = True
                    target = 0
                        
                if ((player_choice == 'bat') or (compChoice == 'ball')) and (not target_set):
                    if (player_move != computer_score):
                        print('initial target:', target)
                        target = int(target) + int(player_move) 
                        print('current target:', target)
                    elif (not target_iteration):
                        print('Target value set:', target)
                        target_set = True
                        #target_iteration = True
                        print('Now the player will Ball and the Computer will Bat')
                        player_choice = 'ball'
                        compChoice = 'bat'

                elif ((compChoice == 'bat') or (player_choice == 'ball')) and (not target_set):
                    if (computer_score != player_move):
                        print('initial target:', target)
                        target = int(target) + int(computer_score)
                        print('current target:', target)
                    elif (not target_iteration):
                        print('Target value set:', target)
                        target_set = True
                        #target_iteration = True
                        print('Now the player will Bat and the computer will Ball')
                        player_choice = 'bat'
                        compChoice = 'ball'

                elif ((player_choice == 'bat') or (compChoice == 'ball')) and target_set and not target_reached:
                    if (player_move != computer_score):
                        target = int(target) - int(player_move)
                        print('Target left:', target)
                        if target < 0:
                            print('Player reached the target\nPlayer Wins!!!')
                    #elif (not target_iteration):
                    else:
                        print('Player Out :(\nComputer Wins!!!')
                        target_reached = True
                        
                elif ((compChoice == 'bat') or (player_choice == 'ball')) and target_set and not target_reached:
                    if(computer_score != player_move):
                        target = int(target) - int(computer_score)
                        print('Target left:', target)
                        if target < 0:
                            print('Computer reached the target\nComputer Wins!!!')
                    #elif (not target_iteration):
                    else:
                        print('Computer Out :(\nPlayer Wins!!!')
                        target_reached = True
                        
        if target_iteration:
            target_iteration = False
            
        if (k == 27) or (target < 0) or target_reached:  # Press ESC to Exit
            print('Game Over!\nHave a nice day ahead :)')
            delete_csv_file()  
            break

        if paused:
            cv2.putText(frame, f"Player's Move: {player_move}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Computer's Move: {computer_score}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Current Target: {target}", (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
            if 'choice' in locals():
                cv2.putText(frame, f"Your Choice: {choice}", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if countdown_timer > 0:
                countdown_timer -= 1

        elif not paused:
            if elapsed_time < pause_interval:
                cv2.putText(frame, f"Countdown: {int(pause_interval - elapsed_time)}", (10, 1250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            else:
                cv2.putText(frame, f"Pls Play Your Turn :)", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Hand Cricket', frame)

    camera_video.release()
    print('Video Released')

    cv2.destroyAllWindows()
    for i in range(1, 5):
        cv2.waitKey(1)
    print('Window Destroyed')

if __name__ == "__main__":
    play_hand_cricket()


Enter 'o' for odd or 'e' for even:  e


Player has chosen Even
Paused
Player's move: 3
Computer's move: 5
Total Score: 8
Player Wins!


Enter "bat" for Batting and "ball" for Balling:  bat


Player has chosen Batting
initial target: 0
current target: 3
Resumed
Paused
Player's move: 3
Computer's move: 4
initial target: 3
current target: 6
Resumed
Paused
Player's move: 4
Computer's move: 1
initial target: 6
current target: 10
Resumed
Paused
Player's move: 5
Computer's move: 2
initial target: 10
current target: 15
Resumed
Paused
Player's move: 2
Computer's move: 1
initial target: 15
current target: 17
Resumed
Paused
Player's move: 5
Computer's move: 4
initial target: 17
current target: 22
Resumed
Paused
Player's move: 2
Computer's move: 6
initial target: 22
current target: 24
Resumed
Paused
Player's move: 5
Computer's move: 4
initial target: 24
current target: 29
Resumed
Paused
Player's move: 2
Computer's move: 3
initial target: 29
current target: 31
Resumed
Paused
Player's move: 4
Computer's move: 6
initial target: 31
current target: 35
Resumed
Paused
Player's move: 3
Computer's move: 5
initial target: 35
current target: 38
Resumed
Paused
Player's move: 4
Computer's move: 3
